Загрузка предобученных эмбеддингов (тренирововчных и тестовых)

In [ ]:
import pandas as pd

embeddings = pd.read_pickle('train_mles_embeddings.pickle')
test_embeddings = pd.read_pickle('test_mles_embeddings.pickle')

In [ ]:
gowalla_all_ids = pd.read_pickle('gowalla_all_ids.pkl')

In [ ]:
import torch
import numpy as np

In [ ]:
from torch import nn
num_classes = 186

Модель, обучающаяся на целевую задачу на предобученных эмбеддингах

In [ ]:
class DownstreamModel(nn.Module):
  def __init__(self):
    super(DownstreamModel, self).__init__()
    self.fc1 = nn.Linear(128, 186)
    self.fc2 = nn.Linear(186, 186)
    self.fc3 = nn.Linear(500, 3883)
    self.fc4 = nn.Linear(186, 186)
    self.dropout = nn.Dropout(p=0.3)
    self.relu = nn.ReLU()

  def forward(self, x):
    x = self.fc1(x)
    x = self.fc2(x)
    return x

In [ ]:
user_target = pd.read_csv('user_activity_target.csv')

In [ ]:
user_activity = pd.read_csv('gowalla_user_activity.csv')

In [ ]:
true_pred  = [gowalla_all_ids[1][x] for x in target]

Метрики

In [ ]:
def apk(actual, predicted, k=10):

    if len(predicted) > k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0

    for i, p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i + 1.0)

    if not actual:
        return 0.0

    return score / min(len(actual), k)


def mapk(y_prob, y, k=10):
    predicted = [np.argsort(p_)[-k:][::-1] for p_ in y_prob]
    actual = [[y_] for y_ in y]
    return np.mean([apk(a, p, k) for a, p in zip(actual, predicted)])



def hits_k(y_prob, y, k=10):
    acc = []
    for p_, y_ in zip(y_prob, y):
        top_k = p_.argsort()[-k:][::-1]
        acc += [1. if y_ in top_k else 0.]
    return sum(acc) / len(acc)

In [ ]:
from torch.utils.data import Dataset, DataLoader

Загрузка эмбеддингов в датасет

In [ ]:
class UserDataset():

    def __init__(self):
        embeddings = pd.read_pickle(r'train_gpt_embeddings.pickle')
        embeddings_tensor = torch.tensor(embeddings.values, requires_grad = False)
        self.x = embeddings_tensor

        user_target = pd.read_csv('user_activity_target.csv')
        target = list(user_target['target_train'])
        true_pred  = [gowalla_all_ids[1][x] for x in target]
        self.y = true_pred


    def __getitem__(self, index):
        return self.x[index], self.y[index]

    def __len__(self):
        return len(self.x)

In [ ]:
class RandomDataset():
    def __init__(self):
        embeddings_tensor = torch.Tensor(size=[20001, 128])
        self.x = embeddings_tensor.random_(-1, 1)
        user_target = pd.read_csv('user_activity_target.csv')
        target = list(user_target['target_train'])
        true_pred  = [gowalla_all_ids[1][x] for x in target]
        self.y = true_pred


    def __getitem__(self, index):
        return self.x[index], self.y[index]

    def __len__(self):
        return len(self.x)


In [ ]:
class UserTestDataset():
    def __init__(self):
        embeddings = pd.read_pickle(r'test_gpt_embeddings.pickle')
        embeddings_tensor = torch.tensor(embeddings.values,requires_grad = False)
        self.x = embeddings_tensor

        user_target = pd.read_csv('user_activity_target.csv')
        target = list(user_target['target_test'])
        true_pred  = [gowalla_all_ids[1][x] for x in target]
        self.y = true_pred


    def __getitem__(self, index):
        return self.x[index], self.y[index]

    def __len__(self):
        return len(self.x)

In [ ]:
train_dataset = UserDataset()
print(train_dataset.__len__())

20001


In [ ]:
test_dataset = UserTestDataset()
print(test_dataset.__len__())

20001


In [ ]:
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=False)

In [ ]:
test_dataloader = DataLoader(test_dataset, batch_size = 64, shuffle = False)

In [ ]:
from sklearn.metrics import top_k_accuracy_score
from sklearn.metrics import ndcg_score

In [ ]:
labels = np.array(gowalla_all_ids[1].values())

In [ ]:
labels = np.arange(0, num_classes)

In [ ]:
def get_metrics_(probs, labels_batch, test_one_hot):
    hits1 = top_k_accuracy_score(labels_batch, probs.cpu().detach().numpy(), k=1, labels = labels)
    hits5 = top_k_accuracy_score(labels_batch, probs.cpu().detach().numpy(), k=5, labels = labels)
    hits10 = top_k_accuracy_score(labels_batch, probs.cpu().detach().numpy(), k=10, labels = labels)
    hits20 = top_k_accuracy_score(labels_batch, probs.cpu().detach().numpy(), k=20, labels = labels)
    hits50= top_k_accuracy_score(labels_batch, probs.cpu().detach().numpy(), k=50, labels = labels)
    hits100 = top_k_accuracy_score(labels_batch, probs.cpu().detach().numpy(), k=100, labels = labels)

    map1 = mapk(y_prob=probs.cpu().detach().numpy(), y = labels_batch, k=1)
    map5 = mapk(y_prob=probs.cpu().detach().numpy(), y = labels_batch, k=5)
    map10 = mapk(y_prob=probs.cpu().detach().numpy(), y = labels_batch, k=10)
    map20 = mapk(y_prob=probs.cpu().detach().numpy(), y = labels_batch, k=20)
    map50 = mapk(y_prob=probs.cpu().detach().numpy(), y = labels_batch, k=50)
    map100 = mapk(y_prob=probs.cpu().detach().numpy(), y = labels_batch, k=100)

    ndcg1 = ndcg_score(test_one_hot, probs.cpu().detach().numpy(), k=1)
    ndcg5 = ndcg_score(test_one_hot, probs.cpu().detach().numpy(), k=5)
    ndcg10 = ndcg_score(test_one_hot, probs.cpu().detach().numpy(), k=10)
    ndcg20 = ndcg_score(test_one_hot, probs.cpu().detach().numpy(), k=20)
    ndcg50 = ndcg_score(test_one_hot, probs.cpu().detach().numpy(), k=50)
    ndcg100 = ndcg_score(test_one_hot, probs.cpu().detach().numpy(), k=100)
    return hits1, hits5, hits10, hits20, hits50, hits100, map1, map5, map10, map20, map50, map100, ndcg1, ndcg5, ndcg10, ndcg20, ndcg50, ndcg100

In [ ]:
user_target = pd.read_csv('user_activity_target.csv')
user_target

,Unnamed: 0,index,user,target_test,target_train
0,0,0,0,16,80
1,1,1,1,35,185
2,2,2,2,99,99
3,3,3,3,127,127
4,4,4,4,169,169
...,...,...,...,...,...
19996,19996,19996,19996,184,184
19997,19997,19997,19997,82,19
19998,19998,19998,19998,182,182
19999,19999,19999,19999,107,107


Обучение и тестирование

In [ ]:
def test(model, test_dataloader, loss_fn):
    metrics_val = []
    model.eval()

    for test_emb_batch, test_labels_batch in test_dataloader:

        test_probs = model(test_emb_batch)

        test_pred = torch.randint(0, num_classes, (len(test_emb_batch),))
        test_one_hot = torch.zeros(len(test_emb_batch), num_classes)
        test_one_hot[torch.arange(len(test_one_hot)), test_labels_batch] = 1
        loss = loss_fn(test_probs, test_one_hot)

        hits1, hits5, hits10, hits20, hits50, hits100, map1, map5, map10, map20, map50, map100, ndcg1, ndcg5, ndcg10, ndcg20, ndcg50, ndcg100 \
        = get_metrics_(test_probs, test_labels_batch, test_one_hot)
        metrics_val.append([hits1, hits5, hits10, hits20, hits50, hits100, map1, map5, map10, map20, map50, map100, ndcg1, ndcg5, ndcg10, ndcg20, ndcg50, ndcg100])


    mean = torch.Tensor(metrics_val).mean(axis=0)
    test_hits1, test_hits5, test_hits10, test_hits20, test_hits50, test_hits100, test_map1, test_map5, test_map10, test_map20, test_map50, test_map100, \
    test_ndcg1, test_ndcg5, test_ndcg10, test_ndcg20, test_ndcg50, test_ndcg100 = mean
    return test_hits1, test_hits5, test_hits10, test_hits20, test_hits50, test_hits100, test_map1, test_map5, test_map10, test_map20, test_map50, test_map100, \
    test_ndcg1, test_ndcg5, test_ndcg10, test_ndcg20, test_ndcg50, test_ndcg100

In [ ]:
def print_metrics(hits1, hits5, hits10, hits20, hits50, hits100, map1, map5, map10, map20, map50, map100, ndcg1, ndcg5, ndcg10, ndcg20, ndcg50, ndcg100):
    print(f'hits@1: {hits1:.6f}, hits@5: {hits5:.6f}, hits@10: {hits10:.6f}, hits@20: {hits20:.6f}')
    print(f'hits@50: {hits50:.6f}, hits@100: {hits100:.6f}')
    print(f'map@1: {map1:.6f}, map@5: {map5:.6f}, map@10: {map10:.6f}, map@20: {map20:.6f}')
    print(f'map@50: {map50:.6f}, map@100: {map100:.6f}')
    print(f'ndcg@1: {ndcg1:.6f}, ndcg@5: {ndcg5:.6f}, ndcg@10: {ndcg10:.6f}, ndcg@20: {ndcg20:.6f}')
    print(f'ndcg@50: {ndcg50:.6f}, ndcg@100: {ndcg100:.6f}')

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
torch.manual_seed(100)

In [ ]:
loss_fn = nn.CrossEntropyLoss()
model  = DownstreamModel()

model.train()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)
for i in range(200):
    hits_1 = []
    losses = []
    print(i)
    for emb_batch, labels_batch in train_dataloader:
        optimizer.zero_grad()
        probs = model(emb_batch)
        pred = torch.argmax(probs, axis = 1)
        one_hot = torch.zeros(len(emb_batch), 186)
        one_hot[torch.arange(len(one_hot)), labels_batch] = 1
        hits_1.append(top_k_accuracy_score(labels_batch, probs.cpu().detach().numpy(), k=1, labels = labels))
        loss = loss_fn(probs, one_hot)
        losses.append(loss)
        loss.backward()
        optimizer.step()
    mean_loss = torch.Tensor(losses).mean().item()
    mean_hits = torch.Tensor(hits_1).mean().item()
    print(f'Loss: {mean_loss}, Hits@1: {mean_hits}')

0
Loss: 4.549046039581299, Hits@1: 0.04447278380393982
1
Loss: 4.251651763916016, Hits@1: 0.05889970064163208
2
Loss: 4.208664417266846, Hits@1: 0.06269362568855286
3
Loss: 4.189017295837402, Hits@1: 0.06334259361028671
4
Loss: 4.176349639892578, Hits@1: 0.06374195218086243
5
Loss: 4.166793346405029, Hits@1: 0.06419122964143753
6
Loss: 4.158962726593018, Hits@1: 0.06474035233259201
7
Loss: 4.152222156524658, Hits@1: 0.06494003534317017
8
Loss: 4.1462321281433105, Hits@1: 0.06518963724374771
9
Loss: 4.1407952308654785, Hits@1: 0.06543923914432526
10
Loss: 4.135788440704346, Hits@1: 0.06553907692432404
11
Loss: 4.131124496459961, Hits@1: 0.06603828072547913
12
Loss: 4.126747131347656, Hits@1: 0.06628787517547607
13
Loss: 4.122610569000244, Hits@1: 0.06648755818605423
14
Loss: 4.118682861328125, Hits@1: 0.066637322306633
15
Loss: 4.114938735961914, Hits@1: 0.06688691675662994
16
Loss: 4.11135721206665, Hits@1: 0.0670865997672081
17
Loss: 4.107922077178955, Hits@1: 0.06728628277778625
18
L

In [ ]:
test_hits1, test_hits5, test_hits10, test_hits20, test_hits50, test_hits100, test_map1, test_map5, test_map10, test_map20, test_map50, test_map100, test_ndcg1, test_ndcg5, test_ndcg10, test_ndcg20, test_ndcg50, test_ndcg100 = test(model, test_dataloader, loss_fn)

print_metrics(test_hits1, test_hits5, test_hits10, test_hits20, test_hits50, test_hits100, test_map1, test_map5, test_map10, test_map20, test_map50, test_map100, test_ndcg1, test_ndcg5, test_ndcg10, test_ndcg20, test_ndcg50, test_ndcg100)

hits@1: 0.036339, hits@5: 0.165708, hits@10: 0.298277, hits@20: 0.488046
hits@50: 0.792738, hits@100: 0.976587
map@1: 0.036339, map@5: 0.078160, map@10: 0.095441, map@20: 0.108497
map@50: 0.118162, map@100: 0.120916
ndcg@1: 0.036339, ndcg@5: 0.099614, ndcg@10: 0.142073, ndcg@20: 0.189910
ndcg@50: 0.250296, ndcg@100: 0.280455
